##### DATASET ACCESS

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df1 = pd.read_csv("aapl.us.txt")
df2 = pd.read_csv("avgo.us.txt")
df3 = pd.read_csv("googl.us.txt")
df4 = pd.read_csv("msft.us.txt")
df5 = pd.read_csv("nvda.us.txt")

In [3]:
df4.tail()

,Date,Open,High,Low,Close,Volume,OpenInt
7978,2017-11-06,84.20,84.700,84.0825,84.47,19852151,0
7979,2017-11-07,84.77,84.900,83.9300,84.26,17927878,0
7980,2017-11-08,84.14,84.610,83.8300,84.56,18029584,0
7981,2017-11-09,84.11,84.270,82.9000,84.09,21175384,0
7982,2017-11-10,83.79,84.095,83.2300,83.87,19396301,0


In [4]:
df1["Ticker"] = ["AAPL"]*len(df1)
df2["Ticker"] = ["AVGO"]*len(df2)
df3["Ticker"] = ["GOOGL"]*len(df3)
df4["Ticker"] = ["MSFT"]*len(df4)
df5["Ticker"] = ["NVDA"]*len(df5)
# Add Ticker column

In [5]:
df = pd.concat([df1,df2,df3,df4,df5], ignore_index=True)
# Create dataframe containing all 5 stocks
df.isna().sum()
# No missing values

Date       0
Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
Ticker     0
dtype: int64

In [6]:
df.set_index(['Ticker','Date'], inplace=True)
df.sort_index()
# Set Multi-index with Level 0: Ticker and Level 1: Date
df.describe()

,Open,High,Low,Close,Volume,OpenInt
count,26496.000000,26496.000000,26496.000000,26496.000000,2.649600e+04,26496.0
mean,71.865779,72.559708,71.106310,71.852050,6.168198e+07,0.0
std,151.158730,152.300934,149.857304,151.122148,7.560150e+07,0.0
min,0.067200,0.067200,0.067200,0.067200,0.000000e+00,0.0
25%,1.969600,1.996000,1.937100,1.970500,1.165063e+07,0.0
50%,18.866000,19.194000,18.589000,18.901000,4.119017e+07,0.0
75%,50.038500,50.548750,49.562500,50.029500,8.383205e+07,0.0
max,1050.050000,1063.620000,1047.050000,1058.290000,2.069770e+09,0.0


##### DATA CLEANING

In [7]:
df.isna().sum()
# No missing values

Open       0
High       0
Low        0
Close      0
Volume     0
OpenInt    0
dtype: int64

In [8]:
df.reset_index(inplace=True)
# We revert the indices to make changes in "Date"

In [9]:
df.Date = pd.to_datetime(df.Date)
# Convert "Date" to datetime

In [10]:
df.set_index(['Ticker','Date'],inplace=True)
df.sort_index(inplace=True)
# Set the indices again.

In [11]:
df.info()
# Check if successful.

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 26496 entries, ('AAPL', Timestamp('1984-09-07 00:00:00')) to ('NVDA', Timestamp('2017-11-10 00:00:00'))
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Open     26496 non-null  float64
 1   High     26496 non-null  float64
 2   Low      26496 non-null  float64
 3   Close    26496 non-null  float64
 4   Volume   26496 non-null  int64  
 5   OpenInt  26496 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 1.6+ MB


In [12]:
dates = [x[1] for x in df.index]
df = df.loc[np.array(dates)>(max(dates)-pd.Timedelta(days=3653))]
# Take max date and subtract 10 years (2008,2012,2016 leap) and create filtered dataframe.

##### DATA TRANSFORMATION

In [22]:
df['Daily Return'] = df.groupby('Ticker')['Close'].pct_change()*100
# Groups by Tickers and gives fractional change * 100.

In [28]:
df

Open     High      Low    Close     Volume  OpenInt  \
Ticker Date                                                                 
AAPL   2007-11-12   21.130   21.479   19.291   19.691  492362604        0   
       2007-11-13   20.615   21.897   19.691   21.765  484373501        0   
       2007-11-14   22.733   22.739   20.970   21.274  403585172        0   
       2007-11-15   21.280   21.717   20.528   21.040  414487458        0   
       2007-11-16   21.193   21.388   20.405   21.309  385660112        0   
...                    ...      ...      ...      ...        ...      ...   
NVDA   2017-11-06  207.200  209.980  206.700  209.630    9731783        0   
       2017-11-07  210.550  212.900  210.056  212.000   10671815        0   
       2017-11-08  211.850  212.000  207.240  209.160   13033902        0   
       2017-11-09  205.270  206.330  200.370  205.320   23895006        0   
       2017-11-10  213.080  218.670  211.630  216.140   31300857        0   

                   Daily Return  
Ticker Date                      
AAPL   2007-11-12           NaN  
       2007-11-13     10.532731  
       2007-11-14     -2.255915  
       2007-11-15     -1.099934  
       2007-11-16      1.278517  
...                         ...  
NVDA   2017-11-06      0.450429  
       2017-11-07      1.130563  
       2017-11-08     -1.339623  
       2017-11-09     -1.835915  
       2017-11-10      5.269823  

[12159 rows x 7 columns]

In [38]:
df['7 Day Moving Average'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(7).mean())
df['30 Day Moving Average'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(30).mean())
# Calculating rolling avgs of 7 days and 30 days by grouping, rolling then mean method.

In [42]:
df['Rolling Volatility (30d)'] = df.groupby('Ticker')['Close'].transform(lambda x: x.rolling(30).std(ddof=0))
# Calculating rolling std of 30 days by grouping, rolling then std method.
# Setting ddof=0 as we need 30 days data only and we have that.

In [45]:
df
# Confirming that operations performed are successful.

Open     High      Low    Close     Volume  OpenInt  \
Ticker Date                                                                 
AAPL   2007-11-12   21.130   21.479   19.291   19.691  492362604        0   
       2007-11-13   20.615   21.897   19.691   21.765  484373501        0   
       2007-11-14   22.733   22.739   20.970   21.274  403585172        0   
       2007-11-15   21.280   21.717   20.528   21.040  414487458        0   
       2007-11-16   21.193   21.388   20.405   21.309  385660112        0   
...                    ...      ...      ...      ...        ...      ...   
NVDA   2017-11-06  207.200  209.980  206.700  209.630    9731783        0   
       2017-11-07  210.550  212.900  210.056  212.000   10671815        0   
       2017-11-08  211.850  212.000  207.240  209.160   13033902        0   
       2017-11-09  205.270  206.330  200.370  205.320   23895006        0   
       2017-11-10  213.080  218.670  211.630  216.140   31300857        0   

                   Daily Return  7 Day Moving Average  30 Day Moving Average  \
Ticker Date                                                                    
AAPL   2007-11-12           NaN                   NaN                    NaN   
       2007-11-13     10.532731                   NaN                    NaN   
       2007-11-14     -2.255915                   NaN                    NaN   
       2007-11-15     -1.099934                   NaN                    NaN   
       2007-11-16      1.278517                   NaN                    NaN   
...                         ...                   ...                    ...   
NVDA   2017-11-06      0.450429            206.281429             192.363667   
       2017-11-07      1.130563            207.730000             193.698333   
       2017-11-08     -1.339623            208.490000             194.812667   
       2017-11-09     -1.835915            208.277143             195.800667   
       2017-11-10      5.269823            209.554286             197.046333   

                   Rolling Volatility (30d)  
Ticker Date                                  
AAPL   2007-11-12                       NaN  
       2007-11-13                       NaN  
       2007-11-14                       NaN  
       2007-11-15                       NaN  
       2007-11-16                       NaN  
...                                     ...  
NVDA   2017-11-06                 10.907855  
       2017-11-07                 10.778489  
       2017-11-08                 10.589656  
       2017-11-09                 10.131281  
       2017-11-10                 10.257322  

[12159 rows x 10 columns]

##### EXPLORATORY DATA ANALYSIS (EDA)

In [50]:
df.groupby('Ticker')['Daily Return'].mean()
# Gives average return over period of 10 years for all tickers.
# Clearly AVGO (Company - Broadcom Inc) has the highest value here.

Ticker
AAPL     0.106375
AVGO     0.166397
GOOGL    0.064454
MSFT     0.058938
NVDA     0.127304
Name: Daily Return, dtype: float64

In [51]:
df.groupby('Ticker')['Rolling Volatility (30d)'].max()
# Gives maximum value of volatility of any month in a given ticker.
# Clearly GOOGL (Company - Alphabet) had the most volatile month.

Ticker
AAPL      7.813506
AVGO     12.243100
GOOGL    59.786266
MSFT      3.577111
NVDA     17.607188
Name: Rolling Volatility (30d), dtype: float64